In [1]:
import numpy as np
from wordcloud import WordCloud
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import seaborn as sns
from tqdm import tqdm_notebook
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
